## This is a demo file 

This file shows how to run the simulator on a random circuit created using the utility functions. These circuits contains only the supported gate set. One can control the number of qubits, number of H gates and it's frequency. 

In this demo file, along with the simulation using **PolyQ** simulator, we also simulate the same circuits using Qiskit's Aer and MQT's DDSIM simulators. 

### Variables default definition
`n`: number of qubits

`t`: total number of variables in the equation

`d`: depth of the circuit

`h_prob`: probability of H gates in the generated random circuit

`ivs`: array of input variables, len(ivs) = n

By default, inital_state variables: [x0,x1,x(n-1)]

`ovs`: array of output variables, len(ovs) = n

`wire_array[q]`: represents the variable name on qubit q, only used to get ovs for now.

`terms`: array containing polynomial equation 



In [30]:
# To relad the changes made in imported files automatically
%load_ext autoreload
%autoreload 2

In [54]:
# Using universal gate set {H,Z,CZ,CCZ,T,S,Tdg,Sdg}
from PolyQ.utils.random_circuit_generator_universal import * 
from PolyQ.engine import *
import psutil, time, numpy as np, csv
from qiskit_aer import StatevectorSimulator
import qiskit.qasm2, qiskit.qasm3
from mqt import ddsim
import time, threading, platform, psutil, signal
import gc
from datetime import datetime

In [5]:
# Function to get a random circuit with given number of qubits and H gates
def get_random_circ_h(n: int,h: int, h_prob: float = None):
    if h_prob == None:
        qc, qr, seed = random_circ_h_const(n, h) # has default value set to 0.125
    else:
        qc, qr, seed = random_circ_h_const(n, h, h_prob)
    return qc, qr, seed

In [ ]:
# Example code (rerun this cell multiple times to see the possible random circuits)
qc, qr , seed = get_random_circ_h(3,3,0.1)
print(qc.draw(fold = -1))

     ┌─────┐┌───┐   ┌─────┐      ┌─────┐┌───┐      ┌───┐  ┌───┐ 
q_0: ┤ Tdg ├┤ S ├─■─┤ Sdg ├─■──■─┤ Tdg ├┤ Z ├──■───┤ H ├──┤ H ├─
     └┬───┬┘└───┘ │ └─────┘ │  │ └─────┘└───┘  │   └───┘  └───┘ 
q_1: ─┤ Z ├───────┼─────────■──■──────────■────■────────────────
      ├───┤       │         │  │  ┌───┐   │  ┌───┐┌─────┐┌─────┐
q_2: ─┤ S ├───────■─────────■──■──┤ S ├───■──┤ H ├┤ Tdg ├┤ Sdg ├
      └───┘                       └───┘      └───┘└─────┘└─────┘


In [53]:
# Functions to get statevector using the three simulators: Qiskit Aer, MQT's DDSIM and PolyQ
def get_stvec_poly(qc, n, t, initial_state):
    terms, wire_array, max_new_var = create_poly(qc, n)
    assert t == max_new_var, "Value of 't' != 'max_new_var' from the create_poly function."
    # print("terms are: ", terms) 
    # print("wires are: ", wire_array)
    ovs = [j[-1] for j in wire_array]
    # print("Output variables are: ", ovs)
    ttb = get_truthtable_no_ivs(terms, n, t, initial_state)
    # print("ttb is: ", ttb)
    stvec = get_statevector_file(ttb, n, t, ovs)
    del ttb, terms, wire_array, max_new_var
    return stvec
    # counts = {} # : To-Do

def get_stvec_ddsim(qc):
    backend = ddsim.DDSIMProvider().get_backend("statevector_simulator")
    job = backend.run(qc)
    result = job.result()
    return result.get_statevector()

def get_stvec_aer(qc):
    backend = StatevectorSimulator()
    res = backend.run(qc).result()
    return res.get_statevector()


In [22]:
# Time Calculation for Simulation using polynomial equation
def get_time_poly(qc, n, t, initial_state):
    start_cpu_times = psutil.Process().cpu_times()
    start_time = time.time()
    # When there is no H gate in our circuit
    if n == t : 
        state_vector = np.zeros(1,dtype=complex)
    else:
        state_vector = get_stvec_poly(qc, n, t, initial_state)

    end_cpu_times = psutil.Process().cpu_times()
    end_time = time.time()

    # Calculate user and system CPU times
    user_time = end_cpu_times.user - start_cpu_times.user
    system_time = end_cpu_times.system - start_cpu_times.system
    cpu_time = user_time + system_time
    wall_time = end_time - start_time

    return (state_vector, cpu_time, wall_time)

# Time Calculation for Simulation using DDSIM by MQT
def get_time_ddsim(qc):
    start_cpu_times = psutil.Process().cpu_times()
    start_time = time.time()

    state_vector = get_stvec_ddsim(qc)

    end_cpu_times = psutil.Process().cpu_times()
    end_time = time.time()

    # Calculate user and system CPU times
    user_time = end_cpu_times.user - start_cpu_times.user
    system_time = end_cpu_times.system - start_cpu_times.system
    cpu_time = user_time + system_time
    wall_time = end_time - start_time

    return (state_vector, cpu_time, wall_time)

# Time Calculation for Simulation using Qiskit's Aer Simulator
def get_time_aer(qc):
    start_cpu_times = psutil.Process().cpu_times()
    start_time = time.time()

    state_vector = get_stvec_aer(qc)
    # printing the statevector amplitudes with a threshold

    end_cpu_times = psutil.Process().cpu_times()
    end_time = time.time()

    # Calculate user and system CPU times
    user_time = end_cpu_times.user - start_cpu_times.user
    system_time = end_cpu_times.system - start_cpu_times.system
    cpu_time = user_time + system_time
    wall_time = end_time - start_time

    return (state_vector, cpu_time, wall_time)


In [23]:
# Function to save the circuit used for demonstration in qasm2 and qasm3 format
def write_results(qc,n,h,h_prob,seed,result,qasm2_filename,qasm3_filename,results_filename):            
    qc_qasm2 = qiskit.qasm2.dumps(qc)
    qc_qasm3 = qiskit.qasm3.dumps(qc)
    with open(qasm2_filename, 'w') as file:
        file.write(f"The seed for the random circuit generator is: {seed}\n")
        file.write(qc_qasm2)
    with open(qasm3_filename, 'w') as file:
        file.write(f"The seed for the random circuit generator is: {seed}\n")
        file.write(qc_qasm3)
    with open(results_filename, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(result)

In [24]:
# Function to run the simulator with a timehandler on Windows and Linux systems
def timeout_handler(signum, frame):
    raise TimeoutError("Process exceeded time limit")

def execute_with_timeout(timeout, func, *args):
    stop_flag = False
    result = None
    
    # Check if we're on a Unix-like system that supports SIGALRM
    if platform.system() != 'Windows' and hasattr(signal, 'SIGALRM'):
        # Unix-like system - use signal-based timeout
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(timeout)
        
        process = psutil.Process()
        memory_usage = process.memory_info().rss
        print(f"Memory usage before func call: {memory_usage / (1024 * 1024):.2f} MB")

        try:
            result = func(*args)
            memory_usage = process.memory_info().rss
            print(f"Memory usage after getting the stvec: {memory_usage / (1024 * 1024):.2f} MB")
        except TimeoutError:
            stop_flag = True
            result = None
        except Exception as e:
            print(f"Error during function execution: {e}")
            stop_flag = True
            result = None
        finally:
            signal.alarm(0)
    
    else:
        # Windows or system without SIGALRM - use threading-based timeout
        process = psutil.Process()
        memory_usage = process.memory_info().rss
        print(f"Memory usage before func call: {memory_usage / (1024 * 1024):.2f} MB")
        
        # Create a container to hold the result and exception
        result_container = {'result': None, 'exception': None, 'completed': False}
        
        def target():
            try:
                result_container['result'] = func(*args)
                result_container['completed'] = True
            except Exception as e:
                result_container['exception'] = e
                result_container['completed'] = True
        
        thread = threading.Thread(target=target)
        thread.daemon = True
        thread.start()
        thread.join(timeout)
        
        if thread.is_alive():
            # Timeout occurred
            stop_flag = True
            result = None
            # Note: We can't forcibly kill the thread, it will continue running
        elif result_container['exception']:
            print(f"Error during function execution: {result_container['exception']}")
            stop_flag = True
            result = None
        else:
            result = result_container['result']
            memory_usage = process.memory_info().rss
            print(f"Memory usage after getting the stvec: {memory_usage / (1024 * 1024):.2f} MB")
    
    return result, stop_flag

#### Simulating the random quantum circuits with varying number of H gates

Make sure that the directory for saving the results exist.

For this demonstration, we are using the Results/demo directory.

In [25]:
# Past data will be overwritten, so do not use 'w'.
with open('Results/demo/program_data_h.csv', 'a', newline='') as file: 
    writer = csv.writer(file)
    writer.writerow(['n', 'h', 'd', 'g', 't', 'h_prob', 'cpu_time_poly', 'wall_time_poly',
                        'cpu_time_ddsim', 'wall_time_ddsim', 'cpu_time_aer', 'wall_time_aer' ])

Main cell which calls the random circuit generator and run all the three simulators on it saving and checking the results.

Adjust the range values as per your choice. 

Things to keep in mind:
- Aer and DDSIM are limited by memory, so after a certain value of n, the kernel will crash. While PolyQ is not limited by memory because we are saving the final state in [stvec_tmp.txt](Results\demo\stvec_tmp.txt) file.
- On a system with 16GB memory, DDSIM will stop working after n >= 28 and Qiskit's Aer will stop working after n >= 29, while PolyQ will run for upto any number of qubits but h limited to 18 depending on your time limit `timeout`. 
- Since the circuits are random, in the given timeout period, some circuit might run while some not with a given value of n and h.
- All three circuit simulators are limited by the time `timeout`.
- Also, Aer runs on multiple cores while the other twos run on single core so you would see quite difference in wall time and cpu time for Aer. 

In [28]:
# Timeout period in seconds, adjust as per the values of n and h, and convenience. 
timeout = 30 

# Lower h_prob means larger circuit, in general. 
for h_prob in np.arange(0.05, 0.051, 0.025): 
    stop_aer = False
    stop_ddsim = False
    for n in range(20,21):
        stop_poly = False
        for h in range(10,11):
            if n > 30: # set this limit as per the system resources
                stop_aer = True
                stop_ddsim = True
            if stop_poly and stop_aer and stop_ddsim:
                break # so that random circ is not created
            qc, qr, seed = get_random_circ_h(n, h, h_prob)
            # print(qc)
            n = qc.width() 
            h = list(instrct.operation.name for _index, instrct in enumerate(qc.data)).count('h') 
            d = qc.depth()  
            g = gate_counts(qc)  
            t = n + h  
            print(f"Running the circuit for n = {n}, h = {h}, h_prob = {h_prob}, d = {d}, g = {g}, t = {t}...")
            # Initialize the state of the qubits
            initial_state = [0 for _ in range(n)]

            # Timeout for poly computation
            if not stop_poly:
                print("Running PolyQ")
                result, stop_poly = execute_with_timeout(timeout, get_time_poly, qc, n, t, initial_state)
                if stop_poly: 
                    print(f"h = {h}, n = {n}, d = {d}, g = {g}")
                    print(f"PolyQ is stopped after h = {h}, and for above values.")
                (stvec_poly, cpu_time_poly, wall_time_poly) = (None,-1,-1) if stop_poly else result 
            else:
                (stvec_poly, cpu_time_poly, wall_time_poly) = (None,-1,-1)
            print()

            # Timeout for aer computation
            if not stop_aer:
                print("Running aer")
                result, stop_aer = execute_with_timeout(timeout, get_time_aer, qc)
                if stop_aer: 
                    print(f"h = {h}, n = {n}, d = {d}, g = {g}")
                    print(f"Aer is stopped after n = {n}, and for above values.")
                (stvec_aer, cpu_time_aer, wall_time_aer) = (None,-1,-1) if stop_aer else result
            else:
                (stvec_aer, cpu_time_aer, wall_time_aer) = (None,-1,-1)
            print()

            # Timeout for ddsim computation
            if not stop_ddsim:
                print("Running ddsim")
                result, stop_ddsim = execute_with_timeout(timeout, get_time_ddsim, qc)
                if stop_ddsim:
                    print(f"h = {h}, n = {n}, d = {d}, g = {g}")
                    print(f"DDSIM is stopped after n = {n}, and for above values.")
                (stvec_ddsim, cpu_time_ddsim, wall_time_ddsim) = (None,-1,-1) if stop_ddsim else result
            else:
                (stvec_ddsim, cpu_time_ddsim, wall_time_ddsim) = (None,-1,-1)
            print()
            
            # Store the result for the current configuration
            results = [n, h, d, g, t, h_prob, 
                    round(cpu_time_poly, 6), round(wall_time_poly, 6),
                    round(cpu_time_ddsim, 6), round(wall_time_ddsim, 6),
                    round(cpu_time_aer, 6), round(wall_time_aer, 6)]

            # Store the circuit in QASM2 and QASM3 format and write the 'results' in 'results_filename'
            qasm2_filename = f'Results/demo/circuits/qc_qasm2_n{n}_h{h}_h_prob{h_prob}.qasm2'
            qasm3_filename = f'Results/demo/circuits/qc_qasm3_n{n}_h{h}_h_prob{h_prob}.qasm3'
            results_filename = 'Results/demo/program_data_h.csv'
            write_results(qc,n,h,h_prob,seed,results,qasm2_filename,qasm3_filename,results_filename)

            # Delete all the variables and free up the memory for next iteration
            del results, qc, qr, seed, d, g, t, initial_state, stvec_poly, cpu_time_poly, wall_time_poly
            del stvec_aer, cpu_time_aer, wall_time_aer, stvec_ddsim, cpu_time_ddsim, wall_time_ddsim
            gc.collect()

            print()

Running the circuit for n = 20, h = 10, h_prob = 0.05, d = 36, g = 190, t = 30...
Running PolyQ
Memory usage before func call: 161.31 MB
Memory usage after getting the stvec: 161.45 MB

Running aer
Memory usage before func call: 161.43 MB
Memory usage after getting the stvec: 177.96 MB

Running ddsim
Memory usage before func call: 161.95 MB
Memory usage after getting the stvec: 187.48 MB




### Simulate a Qiskit quantum circuit in one step

In [3]:
def format_complex(c):
    real = 0.0 if abs(c.real) < 1e-6 else round(c.real, 3)
    imag = 0.0 if abs(c.imag) < 1e-6 else round(c.imag, 3)
    return f"{real:+.3f}{imag:+.3f}j"

In [4]:
# Function to get a random circuit with given number of qubits and H gates
from PolyQ.utils.random_circuit_generator_universal import * 

def get_random_circ_h(n: int,h: int, h_prob: float = None):
    if h_prob == None:
        qc, qr, seed = random_circ_h_const(n, h) # has default value set to 0.125
    else:
        qc, qr, seed = random_circ_h_const(n, h, h_prob)
    return qc, qr, seed

In [50]:
# I am feeling lazy!
circ, _, _ = get_random_circ_h(3,3,0.1)
print(circ.draw(fold = -1))

                       ┌───┐ ┌───┐ ┌───┐
q_0: ──■───■─────────■─┤ Z ├─┤ Z ├─┤ H ├
       │   │         │ ├───┤┌┴───┴┐└───┘
q_1: ──■───■─────────■─┤ H ├┤ Sdg ├─────
     ┌───┐ │ ┌─────┐ │ ├───┤└─────┘     
q_2: ┤ S ├─■─┤ Sdg ├─■─┤ H ├────────────
     └───┘   └─────┘   └───┘            


In [51]:
from PolyQ.simulation import simulate
st_vec = simulate(circ)
st_vec = [format_complex(c) for c in st_vec]
print(st_vec)

Initial state is: [0, 0, 0]
['+0.354+0.000j', '+0.354+0.000j', '+0.000-0.354j', '+0.000-0.354j', '+0.354+0.000j', '+0.354+0.000j', '+0.000-0.354j', '+0.000-0.354j']


In [55]:
st_vec_aer = get_stvec_aer(circ)
st_vec_aer = np.asarray(st_vec_aer)
st_vec_aer = [format_complex(c) for c in st_vec_aer]
print(st_vec_aer)

['+0.354+0.000j', '+0.354+0.000j', '+0.000-0.354j', '+0.000-0.354j', '+0.354+0.000j', '+0.354+0.000j', '+0.000-0.354j', '+0.000-0.354j']


### Simulating GHZ state

In [56]:
from qiskit import QuantumCircuit
import PolyQ 

# GHZ state: 
# |GHZ⟩ = (|000⟩ + |111⟩) / √2
# Using the property: X = HZH
circ = QuantumCircuit(3)
circ.h(0)
circ.h(1)
circ.cz(0, 1)
circ.h(1)
circ.h(2)
circ.cz(1, 2)
circ.h(2)

print(circ.draw(fold=-1))

st_vec = PolyQ.simulate(circ,0)

st_vec = [format_complex(c) for c in st_vec]
print(st_vec)


     ┌───┐                
q_0: ┤ H ├─■──────────────
     ├───┤ │ ┌───┐        
q_1: ┤ H ├─■─┤ H ├─■──────
     ├───┤   └───┘ │ ┌───┐
q_2: ┤ H ├─────────■─┤ H ├
     └───┘           └───┘
Initial state is: [0, 0, 0]
['+0.707+0.000j', '+0.000+0.000j', '+0.000+0.000j', '+0.000+0.000j', '+0.000+0.000j', '+0.000+0.000j', '+0.000+0.000j', '+0.707+0.000j']


In [57]:
st_vec_aer = get_stvec_aer(circ)
st_vec_aer = np.asarray(st_vec_aer)
st_vec_aer = [format_complex(c) for c in st_vec_aer]
print(st_vec_aer)

['+0.707+0.000j', '+0.000+0.000j', '+0.000+0.000j', '+0.000+0.000j', '+0.000+0.000j', '+0.000+0.000j', '+0.000+0.000j', '+0.707+0.000j']
